In [68]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bsenst/classify_services_model")

model = AutoModelForSequenceClassification.from_pretrained("bsenst/classify_services_model")

In [36]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("LABEL_STUDIO_API_KEY")
secret_value_1 = user_secrets.get_secret("LABEL_STUDIO_URL")
secret_value_2 = user_secrets.get_secret("huggingface_rl_course")

from huggingface_hub import login
login(secret_value_2)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
# https://labelstud.io/guide/sdk.html
! pip install label-studio-sdk

In [103]:
# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = secret_value_1
API_KEY = secret_value_0

# Import the SDK and the client module
from label_studio_sdk import Client

# Connect to the Label Studio API and check the connection
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

project = ls.list_projects()[0] # Get the latest project

tasks = project.get_tasks()

In [58]:
target = [task["annotations"][0]["result"][0]["value"]["choices"][0] for task in tasks]
text = [task["data"]["text"] for task in tasks]

In [98]:
df_parsed = pd.DataFrame()
df_parsed["text"] = [task["data"]["text"] for task in tasks]
df_parsed["labels"] = [1 if task["annotations"][0]["result"][0]["value"]["choices"][0]=="SERVICE" else 0 for task in tasks]
df_parsed.head()

,text,labels
0,Mir laufen die Tränen. 😢 Ein unfassbar lesenwe...,0
1,"Einen Tag, nachdem auf dem Bund-Länder-Gipfel ...",0
2,Wenn der öffentliche Raum durch Eigentumstitel...,0
3,Allein in #Hamburg sind nach bisherigen Angabe...,0
4,JA!#obdachlos #wohnungslos #kaltehilfe __Träge...,1


In [ ]:
df_parsed.to_json("df_parsed.json")

In [83]:
! pip install transformers datasets evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [99]:
from datasets import Dataset

services = Dataset.from_pandas(df_parsed).train_test_split(test_size=0.1)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_services = services.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [101]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "NOSERVICE", 1: "SERVICE"}
label2id = {"NOSERVICE": 0, "SERVICE": 1}

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bsenst/classify_services_model", num_labels=2, id2label=id2label, label2id=label2id)

# https://discuss.huggingface.co/t/how-to-turn-wandb-off-in-trainer/6237
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="classify_services_model_live",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_services["train"],
    eval_dataset=tokenized_services["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Cloning https://huggingface.co/bsenst/classify_services_model_live into local empty directory.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689374,0.500000
2,No log,0.690468,0.500000


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


TrainOutput(global_step=6, training_loss=0.7160180409749349, metrics={'train_runtime': 173.7691, 'train_samples_per_second': 0.414, 'train_steps_per_second': 0.035, 'total_flos': 10359997804800.0, 'train_loss': 0.7160180409749349, 'epoch': 2.0})

In [102]:
trainer.push_to_hub()

Upload file runs/May20_19-38-07_acde1b1f1b7d/events.out.tfevents.1684611493.acde1b1f1b7d.34.18: 100%|#########…

To https://huggingface.co/bsenst/classify_services_model_live
   454582d..a886116  main -> main

To https://huggingface.co/bsenst/classify_services_model_live
   a886116..5a81abb  main -> main



'https://huggingface.co/bsenst/classify_services_model_live/commit/a8861168fc2710e33bda6b9890c1cc98262e142f'